From https://www.tensorflow.org/tutorials/keras/keras_tuner

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
# !git clone https://github.com/keras-team/keras-tuner
!pip install -q -U keras-tuner
import kerastuner as kt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Download and prepare the dataset

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model
When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. 

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

### Instantiate the tuner and perform hypertuning
The Keras Tuner has four tuners available - RandomSearch, Hyperband, BayesianOptimization, and Sklearn. In this tutorial, you use the Hyperband tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')  

Callback to clear the training outputs at the end of every training step.

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.

In [8]:
tuner.search(img_train, label_train,
             epochs = 10,
             validation_data = (img_test, label_test),
             callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



Retrain the model with the optimal hyperparameters from the search.

In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 4:36 - loss: 2.2262 - accuracy: 0.18 - ETA: 12s - loss: 1.1662 - accuracy: 0.6021 - ETA: 7s - loss: 0.9771 - accuracy: 0.658 - ETA: 6s - loss: 0.8906 - accuracy: 0.68 - ETA: 5s - loss: 0.8344 - accuracy: 0.70 - ETA: 4s - loss: 0.7944 - accuracy: 0.72 - ETA: 4s - loss: 0.7617 - accuracy: 0.73 - ETA: 4s - loss: 0.7426 - accuracy: 0.74 - ETA: 3s - loss: 0.7197 - accuracy: 0.74 - ETA: 3s - loss: 0.6990 - accuracy: 0.75 - ETA: 3s - loss: 0.6804 - accuracy: 0.76 - ETA: 3s - loss: 0.6736 - accuracy: 0.76 - ETA: 3s - loss: 0.6601 - accuracy: 0.76 - ETA: 3s - loss: 0.6439 - accuracy: 0.77 - ETA: 2s - loss: 0.6286 - accuracy: 0.77 - ETA: 2s - loss: 0.6231 - accuracy: 0.78 - ETA: 2s - loss: 0.6179 - accuracy: 0.78 - ETA: 2s - loss: 0.6098 - accuracy: 0.78 - ETA: 2s - loss: 0.6036 - accuracy: 0.78 - ETA: 2s - loss: 0.5993 - accuracy: 0.78 - ETA: 2s - loss: 0.5909 - accuracy: 0.79 - ETA:

60000/60000 [==============================] - ETA: 5s - loss: 0.4724 - accuracy: 0.78 - ETA: 3s - loss: 0.3369 - accuracy: 0.87 - ETA: 3s - loss: 0.3689 - accuracy: 0.86 - ETA: 3s - loss: 0.3556 - accuracy: 0.86 - ETA: 3s - loss: 0.3538 - accuracy: 0.86 - ETA: 3s - loss: 0.3519 - accuracy: 0.86 - ETA: 3s - loss: 0.3479 - accuracy: 0.86 - ETA: 3s - loss: 0.3510 - accuracy: 0.86 - ETA: 3s - loss: 0.3492 - accuracy: 0.86 - ETA: 2s - loss: 0.3455 - accuracy: 0.87 - ETA: 3s - loss: 0.3483 - accuracy: 0.87 - ETA: 3s - loss: 0.3468 - accuracy: 0.87 - ETA: 3s - loss: 0.3464 - accuracy: 0.87 - ETA: 3s - loss: 0.3432 - accuracy: 0.87 - ETA: 3s - loss: 0.3403 - accuracy: 0.87 - ETA: 3s - loss: 0.3362 - accuracy: 0.87 - ETA: 2s - loss: 0.3359 - accuracy: 0.87 - ETA: 2s - loss: 0.3367 - accuracy: 0.87 - ETA: 2s - loss: 0.3351 - accuracy: 0.87 - ETA: 2s - loss: 0.3375 - accuracy: 0.87 - ETA: 2s - loss: 0.3377 - accuracy: 0.87 - ETA: 2s - loss: 0.3374 - accuracy: 0.87 - ETA: 2s - loss: 0.3372 - accu

60000/60000 [==============================] - ETA: 5s - loss: 0.2218 - accuracy: 0.90 - ETA: 4s - loss: 0.2563 - accuracy: 0.90 - ETA: 3s - loss: 0.2504 - accuracy: 0.91 - ETA: 3s - loss: 0.2665 - accuracy: 0.90 - ETA: 3s - loss: 0.2717 - accuracy: 0.90 - ETA: 3s - loss: 0.2737 - accuracy: 0.89 - ETA: 3s - loss: 0.2753 - accuracy: 0.89 - ETA: 3s - loss: 0.2773 - accuracy: 0.89 - ETA: 3s - loss: 0.2794 - accuracy: 0.89 - ETA: 3s - loss: 0.2763 - accuracy: 0.89 - ETA: 3s - loss: 0.2772 - accuracy: 0.89 - ETA: 2s - loss: 0.2796 - accuracy: 0.89 - ETA: 2s - loss: 0.2805 - accuracy: 0.89 - ETA: 2s - loss: 0.2791 - accuracy: 0.89 - ETA: 2s - loss: 0.2787 - accuracy: 0.89 - ETA: 2s - loss: 0.2803 - accuracy: 0.89 - ETA: 2s - loss: 0.2782 - accuracy: 0.89 - ETA: 2s - loss: 0.2801 - accuracy: 0.89 - ETA: 2s - loss: 0.2788 - accuracy: 0.89 - ETA: 2s - loss: 0.2782 - accuracy: 0.89 - ETA: 2s - loss: 0.2778 - accuracy: 0.89 - ETA: 2s - loss: 0.2782 - accuracy: 0.89 - ETA: 2s - loss: 0.2786 - accu

60000/60000 [==============================] - ETA: 5s - loss: 0.2921 - accuracy: 0.84 - ETA: 3s - loss: 0.2695 - accuracy: 0.89 - ETA: 3s - loss: 0.2469 - accuracy: 0.90 - ETA: 3s - loss: 0.2549 - accuracy: 0.90 - ETA: 3s - loss: 0.2490 - accuracy: 0.90 - ETA: 3s - loss: 0.2446 - accuracy: 0.90 - ETA: 3s - loss: 0.2427 - accuracy: 0.90 - ETA: 3s - loss: 0.2408 - accuracy: 0.90 - ETA: 3s - loss: 0.2400 - accuracy: 0.90 - ETA: 2s - loss: 0.2405 - accuracy: 0.90 - ETA: 2s - loss: 0.2448 - accuracy: 0.90 - ETA: 2s - loss: 0.2494 - accuracy: 0.90 - ETA: 2s - loss: 0.2503 - accuracy: 0.90 - ETA: 2s - loss: 0.2496 - accuracy: 0.90 - ETA: 2s - loss: 0.2499 - accuracy: 0.90 - ETA: 2s - loss: 0.2482 - accuracy: 0.90 - ETA: 2s - loss: 0.2465 - accuracy: 0.90 - ETA: 2s - loss: 0.2467 - accuracy: 0.90 - ETA: 2s - loss: 0.2461 - accuracy: 0.90 - ETA: 2s - loss: 0.2484 - accuracy: 0.90 - ETA: 2s - loss: 0.2484 - accuracy: 0.90 - ETA: 2s - loss: 0.2493 - accuracy: 0.90 - ETA: 2s - loss: 0.2493 - accu

60000/60000 [==============================] - ETA: 4s - loss: 0.2404 - accuracy: 0.90 - ETA: 3s - loss: 0.2466 - accuracy: 0.90 - ETA: 3s - loss: 0.2200 - accuracy: 0.91 - ETA: 3s - loss: 0.2199 - accuracy: 0.91 - ETA: 3s - loss: 0.2321 - accuracy: 0.91 - ETA: 3s - loss: 0.2352 - accuracy: 0.91 - ETA: 3s - loss: 0.2320 - accuracy: 0.91 - ETA: 3s - loss: 0.2298 - accuracy: 0.91 - ETA: 3s - loss: 0.2313 - accuracy: 0.91 - ETA: 3s - loss: 0.2336 - accuracy: 0.91 - ETA: 3s - loss: 0.2363 - accuracy: 0.91 - ETA: 3s - loss: 0.2350 - accuracy: 0.91 - ETA: 3s - loss: 0.2318 - accuracy: 0.91 - ETA: 3s - loss: 0.2339 - accuracy: 0.91 - ETA: 3s - loss: 0.2332 - accuracy: 0.91 - ETA: 2s - loss: 0.2317 - accuracy: 0.91 - ETA: 2s - loss: 0.2306 - accuracy: 0.91 - ETA: 2s - loss: 0.2303 - accuracy: 0.91 - ETA: 2s - loss: 0.2318 - accuracy: 0.91 - ETA: 2s - loss: 0.2321 - accuracy: 0.91 - ETA: 2s - loss: 0.2322 - accuracy: 0.91 - ETA: 2s - loss: 0.2323 - accuracy: 0.91 - ETA: 2s - loss: 0.2333 - accu

The my_dir/intro_to_kt directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional overwrite = True argument while instantiating the tuner.